<a href="https://colab.research.google.com/github/17002235uhi/17002235uhi_DataAnalytics/blob/main/2%20-%20Train_and_Test_Linear_Regressor_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Training and Test a Linear Regressor

# Test and Train Model 1

In [ ]:
# needed to create the data frame
import pandas as pd

# create data frame from csv file we hosted on our github
df = pd.read_csv('https://raw.githubusercontent.com/17002235uhi/17002235uhi_DataAnalytics/main/lineardata1.csv', index_col=0, )

In [ ]:
# make sure we have our data by printing it out
print(df[:6])

   day  mo  temp  NUM_CRIMES
2    2   1  34.0    0.188679
3    3   1  38.2    0.518868
4    4   1  18.2    0.185535
5    5   1   9.3    0.182390
6    6   1   4.4    0.150943
7    7   1   6.5    0.204403


In [ ]:
# needed to help with speedy maths based calculations
import numpy as np

# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df.iloc[np.random.permutation(len(df))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:3]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      day  mo  temp
95      4   4  44.8
649     5  10  50.3
914     4   7  77.6
485     2   4  56.2
993     6   9  74.1
1360    2   9  63.7


In [ ]:
# print out the shuffled data (first 5 rows)
shuffle[:5]

,day,mo,temp,NUM_CRIMES
95,4,4,44.8,0.251572
649,5,10,50.3,0.470270
914,4,7,77.6,0.790123
485,2,4,56.2,0.770270
993,6,9,74.1,0.817901


In [ ]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,3]

# print out the first 6 rows of the targets data.
print(targets[:6])

95      0.251572
649     0.470270
914     0.790123
485     0.770270
993     0.817901
1360    0.517711
Name: NUM_CRIMES, dtype: float64


In [ ]:
SCALE_NUM_CRIMES = 1.0

In [ ]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_CRIMES'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_CRIMES']) - trainsize

# Define the number of input values (predictors)
nppredictors = 3
# Define the number of output values (targets)
noutputs = 1

In [ ]:
# import tensorflow
%tensorflow_version 1.x
import tensorflow as tf

# check the version
print(tf.__version__)

# needed for high-level file management
import shutil  

# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model', ignore_errors=True)

# This is the core of the linear regressor

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_CRIMES, steps=10000)

# check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value to the outputs.
predslistscale = preds['scores']*SCALE_NUM_CRIMES

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Number of Crimes
avg = np.mean(shuffle['NUM_CRIMES'][:trainsize])

# Calculate the RMSE using Number of Crime values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_CRIMES'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

TensorFlow 1.x selected.
1.15.2
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please access pandas data directly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please convert numpy dtypes explicitly.
Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimat

In [ ]:
input = pd.DataFrame.from_dict(data = 
				{'day' : [1,1,1],
         'mo' : [1,1,1],
         'temp' : [61.8, 31.2, 77.0],
        })
					

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)
# Assume number of crimes scale value is 1900 when at a maximum, based on the analysis previously
predslistnorm = preds['scores']
predslistscale = preds['scores']*1900
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0a3f864d50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model/model.ck

In [ ]:
input = pd.DataFrame.from_dict(data = 
				{'day' : [6,6,6],
         'mo' : [1,1,1],
         'temp' : [61.8, 31.2, 77.0],
        })
					

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)
# Assume number of trips scale value is 600000 when at a maximum, based on the analysis from Tutorial 2
predslistnorm = preds['scores']
predslistscale = preds['scores']*1900
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f09e321d250>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model/model.ck

# Test and Train Model 2

In [ ]:
# needed to create the data frame
import pandas as pd

# create data frame from csv file we hosted on our github
df = pd.read_csv('https://raw.githubusercontent.com/17002235uhi/17002235uhi_DataAnalytics/main/lineardata2.csv', index_col=0, )

In [ ]:
# make sure we have our data by printing it out
print(df[:6])

   day  mo  temp  dewp  visib  NUM_CRIMES
2    2   1  34.0  27.9    7.2    0.188679
3    3   1  38.2  36.9    2.4    0.518868
4    4   1  18.2   9.0    9.7    0.185535
5    5   1   9.3  -1.5    9.9    0.182390
6    6   1   4.4  -6.4   10.0    0.150943
7    7   1   6.5  -4.8   10.0    0.204403


In [ ]:
# needed to help with speedy maths based calculations
import numpy as np

# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df.iloc[np.random.permutation(len(df))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:5]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      day  mo  temp  dewp  visib
175     7   6  69.6  51.5   10.0
919     2   7  70.8  53.2   10.0
454     6   3  38.8  27.2   10.0
75      5   3  29.0  10.9   10.0
1435    7  12  34.2  26.2   10.0
1391    5  10  55.2  49.9    5.2


In [ ]:
# print out the shuffled data (first 5 rows)
shuffle[:5]

,day,mo,temp,dewp,visib,NUM_CRIMES
175,7,6,69.6,51.5,10.0,0.836478
919,2,7,70.8,53.2,10.0,0.870370
454,6,3,38.8,27.2,10.0,0.537838
75,5,3,29.0,10.9,10.0,0.361635
1435,7,12,34.2,26.2,10.0,0.517711


In [ ]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,5]

# print out the first 6 rows of the targets data.
print(targets[:6])

175     0.836478
919     0.870370
454     0.537838
75      0.361635
1435    0.517711
1391    0.623978
Name: NUM_CRIMES, dtype: float64


In [ ]:
SCALE_NUM_CRIMES = 1.0

In [ ]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_CRIMES'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_CRIMES']) - trainsize

# Define the number of input values (predictors)
nppredictors = 5
# Define the number of output values (targets)
noutputs = 1

In [ ]:
# import tensorflow
%tensorflow_version 1.x
import tensorflow as tf

# check the version
print(tf.__version__)

# needed for high-level file management
import shutil  

# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model', ignore_errors=True)

# This is the core of the linear regressor

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_CRIMES, steps=10000)

# check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value to the outputs.
predslistscale = preds['scores']*SCALE_NUM_CRIMES

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Number of Crimes
avg = np.mean(shuffle['NUM_CRIMES'][:trainsize])

# Calculate the RMSE using Number of Crime values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_CRIMES'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

1.15.2
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f09e31c3fd0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Ru

In [ ]:
input = pd.DataFrame.from_dict(data = 
				{'day' : [1,1,1],
         'mo' : [1,1,1],
         'temp' : [61.8, 31.2, 77.0],
         'dewp' : [27.9, 36.9, 9.0],
         'visib' : [10.0, 9.8, 7.6 ],
        })
					

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)
# Assume number of crimes scale value is 1900 when at a maximum, based on the analysis previously
predslistnorm = preds['scores']
predslistscale = preds['scores']*1900
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0a3f863a10>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model/model.ck

In [ ]:
input = pd.DataFrame.from_dict(data = 
				{'day' : [6,6,6],
         'mo' : [1,1,1],
         'temp' : [61.8, 31.2, 77.0],
         'dewp' : [27.9, 36.9, 9.0],
         'visib' : [10.0, 9.8, 7.6 ],
        })
					

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)
# Assume number of trips scale value is 600000 when at a maximum, based on the analysis from Tutorial 2
predslistnorm = preds['scores']
predslistscale = preds['scores']*1900
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f09ea896390>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model/model.ck

This test uses day 6 (Friday) instead of day 1 (Sunday) which shows higher number of trips. The other values were left the same.

# Test and Train Model 3

In [ ]:
# needed to create the data frame
import pandas as pd

# create data frame from csv file we hosted on our github
df = pd.read_csv('https://raw.githubusercontent.com/17002235uhi/17002235uhi_DataAnalytics/main/lineardata3.csv', index_col=0, )

In [ ]:
# make sure we have our data by printing it out
print(df[:6])

   day  mo  temp  dewp  visib  wdsp  NUM_CRIMES
2    2   1  34.0  27.9    7.2   5.2    0.188679
3    3   1  38.2  36.9    2.4   5.9    0.518868
4    4   1  18.2   9.0    9.7  14.9    0.185535
5    5   1   9.3  -1.5    9.9  11.5    0.182390
6    6   1   4.4  -6.4   10.0  10.2    0.150943
7    7   1   6.5  -4.8   10.0   8.0    0.204403


In [ ]:
# needed to help with speedy maths based calculations
import numpy as np

# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df.iloc[np.random.permutation(len(df))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:6]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

     day  mo  temp  dewp  visib  wdsp
340    4  12  30.6  12.9   10.0  12.5
118    6   4  48.4  38.2    9.8   6.5
646    2  10  69.4  63.5    6.4   7.4
861    7   5  47.6  36.5    9.9   6.4
26     5   1  33.2  29.6    8.4  11.3
276    3  10  72.8  57.1   10.0  13.1


In [ ]:
# print out the shuffled data (first 5 rows)
shuffle[:5]

,day,mo,temp,dewp,visib,wdsp,NUM_CRIMES
340,4,12,30.6,12.9,10.0,12.5,0.500000
118,6,4,48.4,38.2,9.8,6.5,0.877358
646,2,10,69.4,63.5,6.4,7.4,0.643243
861,7,5,47.6,36.5,9.9,6.4,0.447531
26,5,1,33.2,29.6,8.4,11.3,0.380503


In [ ]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,6]

# print out the first 6 rows of the targets data.
print(targets[:6])

340    0.500000
118    0.877358
646    0.643243
861    0.447531
26     0.380503
276    0.581761
Name: NUM_CRIMES, dtype: float64


In [ ]:
SCALE_NUM_CRIMES = 1.0

In [ ]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_CRIMES'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_CRIMES']) - trainsize

# Define the number of input values (predictors)
nppredictors = 6
# Define the number of output values (targets)
noutputs = 1

In [ ]:
# import tensorflow
%tensorflow_version 1.x
import tensorflow as tf

# check the version
print(tf.__version__)

# needed for high-level file management
import shutil  

# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model', ignore_errors=True)

# This is the core of the linear regressor

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_CRIMES, steps=10000)

# check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value to the outputs.
predslistscale = preds['scores']*SCALE_NUM_CRIMES

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Number of Crimes
avg = np.mean(shuffle['NUM_CRIMES'][:trainsize])

# Calculate the RMSE using Number of Crime values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_CRIMES'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

TensorFlow 1.x selected.
1.15.2
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please access pandas data directly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please convert numpy dtypes explicitly.
Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimat

In [ ]:
input = pd.DataFrame.from_dict(data = 
				{'day' : [1,1,1],
         'mo' : [1,1,1],
         'temp' : [61.8, 31.2, 77.0],
         'dewp' : [27.9, 36.9, 9.0],
         'visib' : [10.0, 9.8, 7.6 ],
         'wdsp' : [5.0, 3.0, 8.0]
        })
					

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)
# Assume number of crimes scale value is 1900 when at a maximum, based on the analysis previously
predslistnorm = preds['scores']
predslistscale = preds['scores']*1900
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8fe4b654d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model/model.ck

In [ ]:
input = pd.DataFrame.from_dict(data = 
				{'day' : [6,6,6],
         'mo' : [1,1,1],
         'temp' : [61.8, 31.2, 77.0],
         'dewp' : [27.9, 36.9, 9.0],
         'visib' : [10.0, 9.8, 7.6 ],
         'wdsp' : [5.0, 3.0, 8.0]
        })
					

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)
# Assume number of trips scale value is 600000 when at a maximum, based on the analysis from Tutorial 2
predslistnorm = preds['scores']
predslistscale = preds['scores']*1900
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8fe4e6bb50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model/model.ck

In [ ]:
plot(y_test, predictions)

NameError: ignored

This test uses day 6 (Friday) instead of day 1 (Sunday) which shows higher number of trips. The other values were left the same.